# Producer - Consumer

# Setup

In [4]:
import threading
import time
import random

from threading import Thread

from queue import Queue

from concurrent.futures import ThreadPoolExecutor

# Logging

In [2]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.debug('Hello')

DEBUG:root:Hello


# Workers
* Our main thread creates some work for a pool of workers to complete

In [ ]:
q = Queue()
results = []

# Creates 25 pieces of work on the queue
def create_work():
    for i in range(25):
        q.put(i)
        
# Works the queue
def do_work():
    while True:
        # Take a number off the queue & multiply by 4
        number = q.get()
        if number is None:
            logger.debug(f'  Thread {threading.current_thread().name} Finished')
            break
            
        logger.debug(f'  Thread {threading.current_thread().name} : Getting Value {number}')
        result = (number, number * 4)
        
        # Apprend to the results
        time.sleep(1)
        results.append(result)
        
        # Signal done
        q.task_done()
   
# Create daemon five worker threads
logger.debug('Main Starting Threads')
workers = 5
threads = [Thread(target=do_work, daemon=True) for i in range(workers)]
[t.start() for t in threads]

# Run the produce function
logger.debug('Main Creating Work')
create_work()

# Join the queue
logger.debug('Main Joining Queue')
q.join()

# Join the threads
logger.debug(results)

# Stop workers
logger.debug('Stopping Workers Threads')
[q.put(None) for i in range(workers)]
[t.join() for t in threads];

# Producer - Consumer
* We extend the pattern
* Now we have a producer that creates a between 1 and 5 jobs
* We have a worker that waits for work
* The worker can do three things before needing a break

## Work Loop

In [31]:
q = Queue()
producing = True

def produce():
    loop = 1
    while producing == True:
        for i in range(random.randint(1,5)):
            work = f'Job {loop}:{i + 1}'
            logger.debug(f'Producing {work}')
            q.put(work)
        loop += 1
        time.sleep(3)

def consume():
    completed = 0
    while True:
        jobs = q.qsize()
        job  = q.get()
        
        if job is None:
            break
        
        logger.debug(f'  Consuming: {job} Remaining: {jobs}')
        q.task_done()
        
        completed += 1
        if completed > 3:
            logger.debug(f'  Consumer: Taking Break')
            time.sleep(2)
            completed = 0

producer = Thread(target=produce)
consumer = Thread(target=consume)

producer.start()
consumer.start()

DEBUG:root:Producing Job 1:1
DEBUG:root:Producing Job 1:2
DEBUG:root:  Consuming: Job 1:1 Remaining: 1
DEBUG:root:  Consuming: Job 1:2 Remaining: 1
DEBUG:root:Producing Job 2:1
DEBUG:root:Producing Job 2:2
DEBUG:root:  Consuming: Job 2:1 Remaining: 0
DEBUG:root:  Consuming: Job 2:2 Remaining: 1
DEBUG:root:  Taking Break
DEBUG:root:Producing Job 3:1
DEBUG:root:  Consuming: Job 3:1 Remaining: 0
DEBUG:root:Producing Job 4:1
DEBUG:root:  Consuming: Job 4:1 Remaining: 0
DEBUG:root:Producing Job 5:1
DEBUG:root:  Consuming: Job 5:1 Remaining: 0


## Cleanup

In [32]:
producing = False
q.put(None)
producer.join()
consumer.join()